In [1]:
import numpy as np  # Librería para aplicar álgebra lineal
import pandas as pd # Para manejar los datos (datasets)

'''from sklearn import preprocessing # Para el procesador de los datos
from sklearn.preprocessing import Imputer # Para adoptar una estrategia para los missing values
from sklearn.preprocessing import LabelEncoder as Codificar # Para codificar variables categóricas
from sklearn.preprocessing import OneHotEncoder # Para pasar el LaberEncoder vector a OneHot matriz
from sklearn.preprocessing import MinMaxScaler # Para realizar el escalado en escala (0-1)
from sklearn.model_selection import train_test_split as Separar # Para dividir en los 2 conjuntos
#from statsmodels.tools.eval_measures import rmse # Para calcular el error
from sklearn.metrics import confusion_matrix as CM # Para construir la matriz de confusión
from matplotlib.colors import ListedColormap as Colors # Para pintar las regiones en Clasificación'''

import matplotlib.pyplot as plt # Para las visualizaciones
import seaborn as sns # Visualizaciones más fancy


#Iremos agregando las bibliotecas a medida que las necesitemos

In [2]:
#Datos de entrada

#Se especifican los tipo de datos para mejorar la performance
eventos = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv', 
                      dtype={"timestamp": object,#intenté hacerla datetime y no anduvo 
                             "event": 'category',
                             "person":object,
                             "url":object,
                             "sku":object,
                             "model":object,
                             "condition":'category',
                             "storage":object,
                             "color":'category',
                             "skus":object,
                             "search_term":object,
                             "staticpage":object,
                             "campaign_source":object,
                             "search_engine":object,
                             "channel":object,
                             "new_vs_returning":'category',
                             "city":object,
                             "region":object,
                             "country":object,
                             "device_type":object,
                             "screen_resolution":object,
                             "operating_system_version":object,
                             "browser_version":object})

labels = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/labels_training_set.csv',
                        dtype={"person":object, "label":bool})

clientesAEvaluar = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv')

In [3]:
#Revisando el formato de los eventos
eventos.head(3)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Confirmando que coinnciden la cantidad de datos a clasificar
print('Cantidad de clientes que no tenemos clasificados:', len(eventos['person'].unique())- len(labels))
print('Cantidad de clientes a clasificar:',len(clientesAEvaluar))

Cantidad de clientes que no tenemos clasificados: 19415
Cantidad de clientes a clasificar: 19415


In [5]:
#Analizando los datos de entrenamiento
print('Cantidades de conversiones:')
print(labels['label'].value_counts())
print('Proporción de conversiones:')
print(labels['label'].value_counts(normalize=True))

Cantidades de conversiones:
False    18434
True       980
Name: label, dtype: int64
Proporción de conversiones:
False    0.949521
True     0.050479
Name: label, dtype: float64


In [6]:
#Agregamos algunos features a los existentes
eventos['timestamp'] =  pd.to_datetime(eventos['timestamp'])
eventos[['marca','modelo']] = eventos['model'].dropna().str.split(' ',n=1,expand=True)
eventos['weekday'] = eventos['timestamp'].dt.day_name()
eventos['hour'] = eventos['timestamp'].dt.hour
eventos['month'] = eventos['timestamp'].dt.month
eventos['day'] = eventos['timestamp'].dt.day

In [31]:
#Creo períodos de tiempo donde ya asignamos a qué mes pertenecen
eventos['principio_de_mes'] = ((eventos['day'] < 10) * eventos['month'])
eventos['fin_de_mes'] = ((eventos['day'] > 20) * eventos['month'])
eventos['mitad_de_mes'] = (((eventos['day'] > 10)  & (eventos['day'] < 21)) * eventos['month'])
#eventos.loc[1:10,['principio_de_mes','mitad_de_mes','fin_de_mes']]

In [30]:
eventos.groupby('person')['principio_de_mes'].value_counts().unstack().reset_index().head().drop(0,axis=1)

principio_de_mes,person,1,2,3,4,5
0,0008ed71,NaN,NaN,NaN,NaN,NaN
1,00091926,NaN,NaN,NaN,NaN,53.0
2,00091a7a,NaN,NaN,NaN,NaN,NaN
3,000ba417,NaN,NaN,NaN,NaN,NaN
4,000c79fe,NaN,NaN,NaN,NaN,NaN


### Vamos creando features: 

In [33]:
#Features en base al total de cada evento
totalEvento = eventos.groupby('person')['event'].value_counts().unstack().reset_index()
totalEvento.rename(lambda x: x if x == 'person' else 'total ' + x, axis = 1, inplace=True)

#Features en base a la frecuencia de cada evento sobre el total (frecuencia relativa)
frecEvento = eventos.groupby('person')['event'].value_counts(normalize=True).unstack().reset_index()
frecEvento.rename(lambda x: x if x == 'person' else 'frec ' + x, axis = 1, inplace=True)

#features en base a la frecuencia relativa de la marca de los dispositivos consultados
frecMarca = eventos.groupby('person')['marca'].value_counts(normalize=True).unstack().reset_index()
frecMarca.rename(lambda x: x if x == 'person' else 'frec ' + x, axis = 1, inplace=True)

#features en base a la cantidad (frecuencia total) de eventos por mes
totalMes = eventos.groupby('person')['month'].value_counts().unstack().reset_index()
totalMes.rename({1:'total_enero',2:'total_febrero',3:'total_marzo',4:'total_abril',5:'total_mayo'}, 
                inplace=True, axis=1)

#features en base a la cantidad de eventos en períodos de aprox 10 días (principio, mitad y fin de mes)
totalPrincipioMes = eventos.groupby('person')['principio_de_mes'].value_counts().unstack().reset_index().head().drop(0,axis=1)
totalPrincipioMes.rename(lambda x: x if x == 'person' else 'total principio_de_mes ' + str(x), axis = 1, inplace=True)

totalMitadMes = eventos.groupby('person')['mitad_de_mes'].value_counts().unstack().reset_index().head().drop(0,axis=1)
totalMitadMes.rename(lambda x: x if x == 'person' else 'total mitad_de_mes ' + str(x), axis = 1, inplace=True)

totalFinMes = eventos.groupby('person')['fin_de_mes'].value_counts().unstack().reset_index().head().drop(0,axis=1)
totalFinMes.rename(lambda x: x if x == 'person' else 'total fin_de_mes ' + str(x), axis = 1, inplace=True)


#features en base a la frecuencia relativa de los dispositivos desde los que se accede
frecDispositivo = eventos.groupby('person')['device_type'].value_counts(normalize=True).unstack().reset_index()
frecDispositivo.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base a la frecuencia relativa de los colores de los teléfonos vistos
frecColor = eventos.groupby('person')['color'].value_counts(normalize=True).unstack().reset_index()
frecColor.rename(lambda x: x if x == 'person' else 'frec color ' + x, axis=1, inplace=True)

#features en base a la frecuencia relativa de los estados de los teléfonos consultados
frecCondicion = eventos.groupby('person')['condition'].value_counts(normalize=True).unstack().reset_index()
frecCondicion.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base al la frecuencia relativa del día de semana en que se consultó
frecDiaSemana = eventos.groupby('person')['weekday'].value_counts(normalize=True).unstack().reset_index().fillna(0)
frecDiaSemana.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base al día (a revisar, capaz convenga hacer algo en base al més y al día, o separar en quincenas)
frecNroDia = eventos.groupby('person')['day'].value_counts(normalize=True).unstack().reset_index()
frecNroDia.rename(lambda x: x if str(x) == 'person' else 'frec dia ' + str(x), axis=1, inplace=True)

#Creamos los features del almacenamiento interno de los dispositivos consultados
frecAlmacenamiento = eventos.groupby('person')['storage'].value_counts(normalize=True).unstack().reset_index()
frecAlmacenamiento.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)


In [42]:
#Empezamos a hacer el merge
#No hay que olvidarse de hacer outer left joins, si no se perderán filas
#Empezamos con features con todas las filas (todos los clientes tienen eventos)
features = frecEvento.merge(totalEvento, how='left', on='person')
features = features.merge(frecMarca, how='left', on='person')
features = features.merge(totalMes, how='left', on='person')
features = features.merge(frecDispositivo, how='left', on='person')
features = features.merge(frecColor, how='left', on='person')
features = features.merge(frecCondicion, how='left', on='person')
features = features.merge(frecDiaSemana, how='left', on='person')
features = features.merge(totalPrincipioMes, how='left', on='person')
features = features.merge(totalMitadMes, how='left', on='person')
features = features.merge(totalFinMes, how='left', on='person')
features = features.merge(frecNroDia, how='left', on='person') #Lo reemplazo por los períodos en los meses?
features = features.merge(frecAlmacenamiento, how='left', on='person')

features.head(2)

,person,frec ad campaign hit,frec brand listing,frec checkout,frec conversion,frec generic listing,frec lead,frec search engine hit,frec searched products,frec staticpage,...,frec dia 30,frec dia 31,frec 128GB,frec 16GB,frec 256GB,frec 32GB,frec 4GB,frec 512MB,frec 64GB,frec 8GB
0,0008ed71,NaN,NaN,0.500000,NaN,0.166667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.666667,NaN,NaN,0.333333,NaN
1,00091926,0.033482,0.055804,0.004464,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.035714,0.128342,0.278075,0.026738,0.352941,NaN,NaN,0.213904,NaN


#### Seguir con feature engeneering, agregando features. Algunas ideas: 
*    Modelo más consultado
*    navegador más usado (capaz ver los 3 o 5 más frecuentes y una columna otros)
*    etc.

## Machine learning
### Primer algoritmo usado: Random Forest

In [35]:
#Cargamos las bibliotecas para usar y validar Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

#Para preparar sets de entrenamiento y prueba
from sklearn.model_selection import train_test_split  

#Nos traemos bibliotecas para medir la performance
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [36]:
'''Tomamos "prestada" una función para presentar prolijamente los n features más importantes.
La función fue tomada casi literal de:
    https://github.com/Featuretools/predict-next-purchase/blob/master/utils.py
'''
def feature_importances(model, features, n=10):
    importances = model.feature_importances_
    zipped = sorted(zip(features, importances), key=lambda x: -x[1])
    for i, f in enumerate(zipped[:n]):
        print("%d: Feature: %s, %.3f" % (i+1, f[0], f[1]))

    return [f[0] for f in zipped[:n]]

In [43]:
#Reemplazo NaNs por ceros
features = features.fillna(0)

#Cargamos X e y para trabajar con Random Forest (y otros algoritmos luego).
X = pd.merge(features, labels, on='person')
X.drop('person', axis=1, inplace=True)
X = X.fillna(0)
y = X.pop('label')

#Ahora hacemos el split de los datos a testear
#Hacemos una división de set de test y entrenamiento, y le ponemos una semilla arbitraria por si queremos replicarlo
seed = 1234
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

In [46]:
#Creamos el clasificador
'''Nota: Tomamos una configuración de random forest encontrada en:
    https://www.datacamp.com/community/tutorials/random-forests-classifier-python'''

clfRF1 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=seed, verbose=0,
            warm_start=False)
#Versión básica y casi default de RF
clfRF2 = RandomForestClassifier(n_estimators=500, random_state=seed)

In [47]:
#Probamos qué tan bien viene clasificando nuestro modelo, usando cross validation con todos los datos
scores = cross_val_score(estimator=clfRF1,X=X, y=y, cv=3, scoring="roc_auc", verbose=True)
print("RF1 AUC %.2f +/- %.2f" % (scores.mean(), scores.std()))

scores = cross_val_score(estimator=clfRF2,X=X, y=y, cv=3, scoring="roc_auc", verbose=True)
print("RF2 AUC %.2f +/- %.2f" % (scores.mean(), scores.std()))

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   49.8s finished


RF1 AUC 0.85 +/- 0.01
RF2 AUC 0.85 +/- 0.01


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.1min finished


In [49]:
#Verificamos la precisión con los datos de entrenamiento
train_RF1 = clfRF1.fit(X_train,y_train)
prediccion = train_RF1.predict_proba(X_test)
precision = accuracy_score(y_test, prediccion[:,[1]] > 0.5) * 100
print("Precisión RF 1 en entrenamiento: %.2f" % precision)
top_features = feature_importances(train_RF1, X.columns, n=20)

scores = cross_val_score(estimator=train_RF1,X=X, y=y, cv=3, scoring="roc_auc", verbose=True)
print("AUC %.2f +/- %.2f" % (scores.mean(), scores.std()))

Precisión RF 1 en entrenamiento: 95.82
1: Feature: frec checkout, 0.039
2: Feature: total checkout, 0.035
3: Feature: total_mayo, 0.029
4: Feature: frec viewed product, 0.022
5: Feature: frec visited site, 0.021
6: Feature: total viewed product, 0.020
7: Feature: frec ad campaign hit, 0.019
8: Feature: frec generic listing, 0.019
9: Feature: frec search engine hit, 0.018
10: Feature: frec Bom, 0.017
11: Feature: frec color Dourado, 0.017
12: Feature: frec color Preto, 0.017
13: Feature: frec brand listing, 0.016
14: Feature: frec 16GB, 0.016
15: Feature: frec Muito Bom, 0.016
16: Feature: frec 32GB, 0.015
17: Feature: total brand listing, 0.014
18: Feature: total ad campaign hit, 0.014
19: Feature: frec Excelente, 0.014
20: Feature: total generic listing, 0.014
AUC 0.85 +/- 0.01


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   54.1s finished


In [51]:
#Verificamos la precisión con los datos de entrenamiento
train_RF2 = clfRF2.fit(X_train,y_train)
prediccion = train_RF2.predict_proba(X_test)
precision = accuracy_score(y_test, prediccion[:,[1]] > 0.5) * 100
print("Precisión RF 1 en entrenamiento: %.2f" % precision)
top_features = feature_importances(train_RF2, X.columns, n=20)

scores = cross_val_score(estimator=train_RF2,X=X, y=y, cv=3, scoring="roc_auc", verbose=True)
print("AUC %.2f +/- %.2f" % (scores.mean(), scores.std()))

Precisión RF 1 en entrenamiento: 95.82
1: Feature: frec checkout, 0.039
2: Feature: total checkout, 0.035
3: Feature: total_mayo, 0.029
4: Feature: frec viewed product, 0.022
5: Feature: frec visited site, 0.021
6: Feature: total viewed product, 0.020
7: Feature: frec ad campaign hit, 0.019
8: Feature: frec generic listing, 0.019
9: Feature: frec search engine hit, 0.018
10: Feature: frec Bom, 0.017
11: Feature: frec color Dourado, 0.017
12: Feature: frec color Preto, 0.017
13: Feature: frec brand listing, 0.016
14: Feature: frec 16GB, 0.016
15: Feature: frec Muito Bom, 0.016
16: Feature: frec 32GB, 0.015
17: Feature: total brand listing, 0.014
18: Feature: total ad campaign hit, 0.014
19: Feature: frec Excelente, 0.014
20: Feature: total generic listing, 0.014
AUC 0.85 +/- 0.01


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.1min finished


In [23]:
#Pruebo de nuevo los random forest con los resultados obtenidos de mejores clasificadores
scores = cross_val_score(estimator=clfRF1,X=X.loc[:,top_features], y=y, cv=3, scoring="roc_auc", verbose=True)

"RF1 %d features más importantes: AUC %.2f +/- %.2f" % (len(top_features), scores.mean(), scores.std())

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   28.8s finished


'AUC 0.81 +/- 0.01'

In [57]:
#Y si subo la cantidad de features importantes?

top_features = feature_importances(train_RF2, X.columns, n=int(len(X.columns) / 4))
scores = cross_val_score(estimator=clfRF1,X=X.loc[:,top_features], y=y, cv=3, scoring="roc_auc", verbose=True)

"RF1 %d features más importantes: AUC %.2f +/- %.2f" % (len(top_features), scores.mean(), scores.std())

1: Feature: frec checkout, 0.039
2: Feature: total checkout, 0.035
3: Feature: total_mayo, 0.029
4: Feature: frec viewed product, 0.022
5: Feature: frec visited site, 0.021
6: Feature: total viewed product, 0.020
7: Feature: frec ad campaign hit, 0.019
8: Feature: frec generic listing, 0.019
9: Feature: frec search engine hit, 0.018
10: Feature: frec Bom, 0.017
11: Feature: frec color Dourado, 0.017
12: Feature: frec color Preto, 0.017
13: Feature: frec brand listing, 0.016
14: Feature: frec 16GB, 0.016
15: Feature: frec Muito Bom, 0.016
16: Feature: frec 32GB, 0.015
17: Feature: total brand listing, 0.014
18: Feature: total ad campaign hit, 0.014
19: Feature: frec Excelente, 0.014
20: Feature: total generic listing, 0.014
21: Feature: total visited site, 0.013
22: Feature: total search engine hit, 0.012
23: Feature: frec 64GB, 0.012
24: Feature: frec Samsung, 0.012
25: Feature: frec Thursday, 0.012
26: Feature: frec color Branco, 0.012
27: Feature: frec searched products, 0.012
28: Fe

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   44.5s finished


'RF1 42 features más importantes: AUC 0.84 +/- 0.00'

In [59]:
#Veamos cuantos predice que convierten en los 15 primeros días de junio el modelo completo
print('total del set de entrenamiento que convirtieron:', labels['label'].sum())

clfRF1.fit(X,y)
prediccionTest = clfRF1.predict(X=X)
print('total del set de entrenamiento que predice que convertirán, prediciendo con set completo:',pd.Series(prediccionTest).sum())

#Veamos cuantos predice que convierten en los 15 primeros días de junio el modelo completo
train_RF2.fit(X_train,y_train)
prediccionTest = train_RF2.predict(X=X)
print('total del set de entrenamiento que predice que convertirán, prediciendo con train_set:',pd.Series(prediccionTest).sum())

total del set de entrenamiento que convirtieron: 980
total del set de entrenamiento que predice que convertirán, prediciendo con set completo: 979
total del set de entrenamiento que predice que convertirán, prediciendo con train_set: 778


## Vamos a probar otro clasificador: XGBoost

In [60]:
#Primero importamos las bibliotecas que necesitaremos
import xgboost as xgb

In [62]:
#Probamos el modelo más sencillo de XGBoost, con los parámetros por defecto
xgbBasico = xgb.XGBClassifier(seed=seed)
train_xgbBasico = xgbBasico.fit(X_train, y_train)

mejor_n_limit = 0
mejor_precision = 0
for i in range(200):
    prediccion = train_xgbBasico.predict_proba(X_test, ntree_limit=i)
    precision = accuracy_score(y_test, prediccion[:,[1]] > 0.5) * 100
    if(precision > mejor_precision):
        mejor_precision = precision
        mejor_n_limit = i
    
print("Precisión trainmodeloBasico con ntree_limit=",mejor_n_limit,": %.2f" % mejor_precision)

scores = cross_val_score(estimator=train_xgbBasico,X=X, y=y, cv=3, scoring="roc_auc", verbose=True)
print("AUC usando el modelo entrenado con set de entrenamiento reducido %.2f +/- %.2f" % (scores.mean(), scores.std()))

xgbBasico.fit(X, y)
scores = cross_val_score(estimator=modeloBasico,X=X, y=y, cv=3, scoring="roc_auc", verbose=True)
print("AUC usando el modelo entrenado con set de entrenamiento completo %.2f +/- %.2f" % (scores.mean(), scores.std()))

Precisión trainmodeloBasico con ntree_limit= 58 : 95.96


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   29.4s finished


AUC usando el modelo entrenado con set de entrenamiento reducido 0.85 +/- 0.00
AUC usando el modelo entrenado con set de entrenamiento completo 0.85 +/- 0.00


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   29.5s finished


In [63]:
#Una de las particularidades de XGBoost es que es los primeros árboles generados suelen ser más importantes
#Entonces, vamos a probar ordenando el set de entrenamiento
X_y = features.merge(labels, on='person', how='inner')
X_y.sort_values(by='label', ascending=False, inplace=True)
X_y.head(2)

,person,frec ad campaign hit,frec brand listing,frec checkout,frec conversion,frec generic listing,frec lead,frec search engine hit,frec searched products,frec staticpage,...,frec dia 31,frec 128GB,frec 16GB,frec 256GB,frec 32GB,frec 4GB,frec 512MB,frec 64GB,frec 8GB,label
14682,c2acc1c5,0.153846,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,1.0,0.1,0.0,0.0,0.1,0.0,0.0,0.8,0.0,True
18466,f3c6e808,0.217391,0.0,0.0,0.0,0.173913,0.0,0.173913,0.086957,0.0,...,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.6,0.0,True


In [65]:
#Creamos los features ordenando por label (los que están en true, primero)
X_ordenado = X_y.drop('person', axis=1)
y_ordenado = X_ordenado.pop('label')

#Ordenamos los sets que tenemos de entrenamiento In [9]: result = pd.concat([df1, df4], axis=1, sort=False)

X_train_ordenado = pd.concat([X_train,y_train], axis=1)
X_train_ordenado.sort_values(by='label', ascending=False, inplace=True)
y_train_ordenado = X_train_ordenado.pop('label')

In [79]:
#Repetimos las pruebas con el set ordenado
#Entonces, vamos a probar ordenando el set de entrenamiento
#Probamos el modelo más sencillo de XGBoost, con los parámetros por defecto
trainOrdenado_xgbBasico= xgbBasico.fit(X_train_ordenado, y_train_ordenado).fit(X_train_ordenado, y_train_ordenado)

mejor_n_limit = 0
mejor_precision = 0
for i in range(200):
    prediccion = trainOrdenado_xgbBasico.predict_proba(X_test, ntree_limit=i)
    precision = accuracy_score(y_test, prediccion[:,[1]] > 0.5) * 100
    if(precision > mejor_precision):
        mejor_precision = precision
        mejor_n_limit = i
    
print("Precisión modeloBasico con ntree_limit=",mejor_n_limit,": %.2f" % mejor_precision)

scores = cross_val_score(estimator=trainOrdenado_xgbBasico,X=X, y=y, cv=3, scoring="roc_auc", verbose=True)
print("AUC usando trainOrdenado_xgbBasico con datos sin ordenar %.2f +/- %.2f" % (scores.mean(), scores.std()))

scores = cross_val_score(estimator=train_modeloBasico,X=X_ordenado, y=y_ordenado, cv=3, scoring="roc_auc", verbose=True)
print("AUC usando trainOrdenado_xgbBasico con datos ordenados %.2f +/- %.2f" % (scores.mean(), scores.std()))

xgbBasicoOrdenado = xgbBasico.fit(X_ordenado, y_ordenado)
scores = cross_val_score(estimator=xgbBasicoOrdenado,X=X_ordenado, y=y_ordenado, cv=3, scoring="roc_auc", verbose=True)
print("AUC usando xgbBasicoOrdenado con datos ordenados %.2f +/- %.2f" % (scores.mean(), scores.std()))

xgbBasico.fit(X, y)
scores = cross_val_score(estimator=xgbBasico,X=X_ordenado, y=y_ordenado, cv=3, scoring="roc_auc", verbose=True)
print("AUC usando el modelo xgbBasico con datos ordenados %.2f +/- %.2f" % (scores.mean(), scores.std()))

Precisión modeloBasico con ntree_limit= 58 : 95.96


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   34.5s finished


AUC usando trainOrdenado_xgbBasico con datos sin ordenar 0.85 +/- 0.00


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   30.9s finished


AUC usando trainOrdenado_xgbBasico con datos ordenados 0.85 +/- 0.01


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   32.2s finished


AUC usando xgbBasicoOrdenado con datos ordenados 0.85 +/- 0.01
AUC usando el modelo xgbBasico con datos ordenados 0.85 +/- 0.01


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   32.2s finished


In [80]:
#Features mas importantes para XGBoost basico?
top_features = feature_importances(xgbBasico, X.columns, n=int(len(X.columns) / 4))

1: Feature: frec checkout, 0.129
2: Feature: total_mayo, 0.077
3: Feature: frec conversion, 0.055
4: Feature: frec Computer, 0.046
5: Feature: frec Smartphone, 0.036
6: Feature: total viewed product, 0.033
7: Feature: frec dia 31, 0.030
8: Feature: frec brand listing, 0.027
9: Feature: frec ad campaign hit, 0.024
10: Feature: frec color Preto, 0.022
11: Feature: frec dia 4, 0.019
12: Feature: frec 8GB, 0.018
13: Feature: frec search engine hit, 0.016
14: Feature: total checkout, 0.016
15: Feature: frec color Preto Matte, 0.016
16: Feature: frec Muito Bom, 0.016
17: Feature: frec searched products, 0.015
18: Feature: total_marzo, 0.015
19: Feature: frec dia 5, 0.015
20: Feature: frec dia 30, 0.015
21: Feature: total visited site, 0.013
22: Feature: frec dia 2, 0.013
23: Feature: frec generic listing, 0.012
24: Feature: frec color Cinza espacial, 0.012
25: Feature: frec Saturday, 0.012
26: Feature: frec dia 6, 0.012
27: Feature: frec 16GB, 0.012
28: Feature: frec 64GB, 0.012
29: Feature:

In [82]:
xgbBasicoOrdenado.fit(X_ordenado.loc[:,top_features], y_ordenado)
scores = cross_val_score(estimator=xgbBasicoOrdenado,X=X_ordenado, y=y_ordenado, cv=3, scoring="roc_auc", verbose=True)
print("AUC usando el modelo xgbBasicoOrdenado entrenado con top features y datos ordenados %.2f +/- %.2f" % (scores.mean(), scores.std()))

AUC usando el modelo xgbBasicoOrdenado entrenado con top features y datos ordenados 0.85 +/- 0.01


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   26.4s finished


## Preparamos los datos que necesitamos clasificar

In [67]:
datosTest = clientesAEvaluar.merge(features, on='person',how='inner')
print('cantidad de datos:',len(datosTest))
datosTest.head(2)

cantidad de datos: 19415


,person,frec ad campaign hit,frec brand listing,frec checkout,frec conversion,frec generic listing,frec lead,frec search engine hit,frec searched products,frec staticpage,...,frec dia 30,frec dia 31,frec 128GB,frec 16GB,frec 256GB,frec 32GB,frec 4GB,frec 512MB,frec 64GB,frec 8GB
0,4886f805,0.000000,0.000000,0.111111,0.0,0.111111,0.000000,0.111111,0.111111,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.0
1,0297fc1e,0.051146,0.007055,0.012346,0.0,0.037037,0.001764,0.000000,0.010582,0.0,...,0.024691,0.007055,0.121655,0.311436,0.014599,0.082725,0.0,0.0,0.469586,0.0


In [68]:
#Verifico que no se hayan "perdido" filas en el camino (un merge mal definido capaz?)
print('cantidad de personas en total:',len(eventos['person'].unique()))
print('cantidad de personas para training:',len(labels))
print('cantidad de personas para evaluar:',len(clientesAEvaluar))
print('cantidad de personas en clientes agrupados:',len(features))

cantidad de personas en total: 38829
cantidad de personas para training: 19414
cantidad de personas para evaluar: 19415
cantidad de personas en clientes agrupados: 38829


In [73]:
#Haciendo las predicciones con el modelo de Random Forest
prediccionRandomForest = clfRF1.predict_proba(X=datosTest.drop('person', axis=1).fillna(0))

In [74]:
for i in range(1,6):
    print('%d\tclientes con p > %.2f de convertir' % ((prediccionRandomForest[:,[1]] > i/10).sum(),i/10))

3731	clientes con p > 0.10 de convertir
1231	clientes con p > 0.20 de convertir
233	clientes con p > 0.30 de convertir
33	clientes con p > 0.40 de convertir
3	clientes con p > 0.50 de convertir


In [76]:
#Haciendo predicciones con el modelo básico de XGBoost entrenado con los datos ordenados por label
'''prediccionXGBoostBasicoOrdenado = modeloBasico.predict_proba(data=datosTest.drop('person', axis=1).fillna(0),
                                                        ntree_limit=mejor_n_limit) 
"Habrá que ver si no "overfittea" al ponerle el mejor n_limit'''

xgbBasico.fit(X_ordenado, y_ordenado)
prediccionXGBoostBasicoOrdenado = xgbBasico.predict_proba(data=datosTest.drop('person', axis=1).fillna(0))

for i in range(1,6):
    print('%d\tclientes con p > %.2f de convertir' % ((prediccionXGBoostBasicoOrdenado[:,[1]] > i/10).sum(),i/10))

3276	clientes con p > 0.10 de convertir
1001	clientes con p > 0.20 de convertir
276	clientes con p > 0.30 de convertir
121	clientes con p > 0.40 de convertir
77	clientes con p > 0.50 de convertir


In [77]:
#Haciendo predicciones con el modelo básico de XGBoost entrenado con los datos sin ordenar

xgbBasico.fit(X, y)
prediccionXGBoostBasico = xgbBasico.predict_proba(data=datosTest.drop('person', axis=1).fillna(0))

for i in range(1,6):
    print('%d\tclientes con p > %.2f de convertir' % ((prediccionXGBoostBasico[:,[1]] > i/10).sum(),i/10))

3276	clientes con p > 0.10 de convertir
1001	clientes con p > 0.20 de convertir
276	clientes con p > 0.30 de convertir
121	clientes con p > 0.40 de convertir
77	clientes con p > 0.50 de convertir


In [78]:
#Si nos quedamos con el modelo de XGBoost
clientesAEvaluar['label'] = prediccionXGBoostBasico[:,[1]]
clientesAEvaluar.head()

,person,label
0,4886f805,0.008334
1,0297fc1e,0.037808
2,2d681dd8,0.015516
3,cccea85e,0.090828
4,4c8a8b93,0.032561


In [83]:
#Exportamos el archivo a subir. De esta manera tiene el formato correcto. 
#Notar que así lo deja en el directorio de la notebook
clientesAEvaluar.to_csv('entrega_XGBoostBasico 3 0.85647.csv',index=False)
#Esta entrega dio 0.85647

#### Con la entrega anterior llegamos a 0.85443.

In [86]:
#Ensamblando resultados de XGBoost y Random Forest
clientesAEvaluar['label'] = (prediccionXGBoostBasico[:,[1]] + prediccionRandomForest[:,[1]])/2
clientesAEvaluar.to_csv('entrega_XGBoostBasico y RF 4 con promedio.csv',index=False)
clientesAEvaluar.head()

,person,label
0,4886f805,0.006167
1,0297fc1e,0.055904
2,2d681dd8,0.011758
3,cccea85e,0.122414
4,4c8a8b93,0.031280


In [85]:
#Otro ensamble, en lugar del promedio, asignarle pesos
clientesAEvaluar['label'] = (prediccionXGBoostBasico[:,[1]] * 0.6 + prediccionRandomForest[:,[1]] * 0.4)
clientesAEvaluar.to_csv('entrega_XGBoostBasico y RF 2.csv',index=False)
clientesAEvaluar.head(2)

,person,label
0,4886f805,0.006600
1,0297fc1e,0.052285


In [56]:
#Otro ensamble, en lugar del promedio, asignarle pesos
clientesAEvaluar['label'] = (prediccionXGBoostBasico[:,[1]] * 0.4 + prediccionRandomForest[:,[1]] * 0.6)
clientesAEvaluar.to_csv('entrega_XGBoostBasico y RF 3.csv',index=False)
clientesAEvaluar.head()
